In [1]:
# import sys
# sys.path # python导入package时自动搜索的路径
# sys.path.append("/root/Docqa")
import sys
sys.path
sys.path.append("/root/")  
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import PyPDFLoader, ArxivLoader,PyPDFDirectoryLoader
from Docqa.text_spliter import ChineseTextSplitter
import PyPDF2 # pdf reader
from pypdf import PdfReader
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
# from langchain.chains import ConversationalRetrievalChain


In [ ]:

loader = Docx2txtLoader('/data/documents/分布式计算系统.docx')
input_doc = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=650, chunk_overlap=0)
texts = text_splitter.split_documents(input_doc)

In [23]:

loader = PyPDFLoader("/data/documents/深度学习_582660_231113.pdf")
pages = loader.load()
pdf_splitter = ChineseTextSplitter(chunk_size=650, chunk_overlap=0, pdf=True)
pdf_text = pdf_splitter.split_documents(pages)


In [3]:
loader = PyPDFLoader("/data/documents/2304.04912.pdf")
pages = loader.load()
pdf_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=165)
pdf_text = pdf_splitter.split_documents(pages)
len(pdf_text)

23

In [4]:
pdf_text

[Document(page_content='Financial Time Series Forecasting using CNN and Transformer\nZhen Zeng, Rachneet Kaur, Suchetha Siddagangappa,\nSaba Rahimi, Tucker Balch, Manuela Veloso\nJ. P. Morgan AI Research, New York, NY , USA\nAbstract\nTime series forecasting is important across various domains\nfor decision-making. In particular, ﬁnancial time series such\nas stock prices can be hard to predict as it is difﬁcult to\nmodel short-term and long-term temporal dependencies be-\ntween data points. Convolutional Neural Networks (CNN) are\ngood at capturing local patterns for modeling short-term de-\npendencies. However, CNNs cannot learn long-term depen-\ndencies due to the limited receptive ﬁeld. Transformers on the\nother hand are capable of learning global context and long-\nterm dependencies. In this paper, we propose to harness the\npower of CNNs and Transformers to model both short-term\nand long-term dependencies within a time series, and forecast\nif the price would go up, down or rem

In [2]:
from bigdl.llm.langchain.embeddings import TransformersEmbeddings

embeddings = TransformersEmbeddings.from_model_id(model_id="/data/vicuna-7b-v1.5")

2024-01-20 01:33:25,676 - WARNING - BigdlNativeEmbeddings has been deprecated, please switch to the new LMEmbeddings API for sepcific models.


In [6]:
from langchain.vectorstores import FAISS

vectorstore_db = FAISS.from_documents(pdf_text, embeddings)
retriever = vectorstore_db.as_retriever(search_type="similarity", search_kwargs={"k": 6})

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2024-01-20 00:44:06,382 - INFO - Loading faiss with AVX2 support.
2024-01-20 00:44:06,432 - INFO - Successfully loaded faiss with AVX2 support.


In [7]:
from langchain.prompts import PromptTemplate
custom_prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context and answer the question at the end.
If you don't know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.
Context= {context}
History = {history}
Question= {question}
Helpful Answer:
"""

prompt = PromptTemplate(template=custom_prompt_template,
                        input_variables=["question", "context", "history"])

In [2]:
from langchain.chains.chat_vector_db.prompts import QA_PROMPT
from langchain.chains.question_answering import load_qa_chain

from bigdl.llm.langchain.llms import TransformersLLM

llm = TransformersLLM.from_model_id(
        model_id="/data/vicuna-7b-v1.5",
        model_kwargs={"temperature": 0, "max_length": 1024, "trust_remote_code": True},
    )

TypeError: issubclass() arg 1 must be a class

In [12]:
qa_chain_with_memory = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff',
                                                   retriever = vectorstore_db.as_retriever(),
                                                   return_source_documents = True,
                                                   chain_type_kwargs = {"verbose": True,
                                                                        "prompt": prompt,
                                                                        "memory": ConversationBufferMemory(
                                                                            input_key="question",
                                                                            memory_key="history",
                                                                            return_messages=True)})


In [13]:
query = "what is the main theory of the paper?"
qa_chain_with_memory({"query": query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant for question-answering tasks. Use the following pieces of retrieved context and answer the question at the end.
If you don't know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.
Context= and long-term dependencies within a time series, and forecast
if the price would go up, down or remain the same (ﬂat) in
the future. In our experiments, we demonstrated the success
of the proposed method in comparison to commonly adopted
statistical and deep learning methods on forecasting intraday
stock price change of S&P 500 constituents.
Introduction
Time series forecasting is challenging, especially in the ﬁ-
nancial industry (Pedersen 2019). It involves statistically
understanding complex linear and non-linear interactions
within historical data to predict the future. I

/root/miniconda3/envs/llm-tutorial/lib/python3.9/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (4096) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


The main theory of the paper is the use of a transformer-based approach for time series forecasting, specifically for intraday stock price change prediction of S&P 500 constituents. The proposed method demonstrates success in comparison to traditional statistical and deep learning methods. The transformer architecture allows for parallel computations and bypasses the recursion issue in RNNs and LSTMs, making it suitable for handling long-term dependencies within a time series.

> Finished chain.

> Finished chain.


{'query': 'what is the main theory of the paper?',
 'result': "You are an assistant for question-answering tasks. Use the following pieces of retrieved context and answer the question at the end.\nIf you don't know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.\nContext= and long-term dependencies within a time series, and forecast\nif the price would go up, down or remain the same (ﬂat) in\nthe future. In our experiments, we demonstrated the success\nof the proposed method in comparison to commonly adopted\nstatistical and deep learning methods on forecasting intraday\nstock price change of S&P 500 constituents.\nIntroduction\nTime series forecasting is challenging, especially in the ﬁ-\nnancial industry (Pedersen 2019). It involves statistically\nunderstanding complex linear and non-linear interactions\nwithin historical data to predict the future. In the ﬁnan-\ncial industry, com

In [14]:
query = "simply introduce the data sets the paper used."
qa_chain_with_memory({"query": query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant for question-answering tasks. Use the following pieces of retrieved context and answer the question at the end.
If you don't know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.
Context= are often correct, thus thresholding ﬁlters out erroneous low-
conﬁdence predictions, leading to a boost in the sign accu-
racy.
Another highlight is the signiﬁcant gap that CTTS has
achieved for thresholded 3-class∗accuracy compared to
other baselines. This can be harnessed in the future for trad-
ing decision-making. For example, a straightforward trad-
ing decision can be buy/sell/hold stocks when the predicted
class is up/down/ﬂat, respectively. Given that CTTS’s pre-
dicted probablitlies are reliable as discussed earlier, the
amount of stock shares to buy/sell/hold can d

/root/miniconda3/envs/llm-tutorial/lib/python3.9/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (4096) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


The paper used licensed Bloomberg data service for intraday stock prices of S&P 500 constituents for the year 2019, sampled at 1-minute interval for 52 weeks with 7 time series for each week. The data was randomly split into training (80%) and validation (20%) sets.

> Finished chain.

> Finished chain.


{'query': 'simply introduce the data sets the paper used.',
 'result': 'You are an assistant for question-answering tasks. Use the following pieces of retrieved context and answer the question at the end.\nIf you don\'t know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.\nContext= are often correct, thus thresholding ﬁlters out erroneous low-\nconﬁdence predictions, leading to a boost in the sign accu-\nracy.\nAnother highlight is the signiﬁcant gap that CTTS has\nachieved for thresholded 3-class∗accuracy compared to\nother baselines. This can be harnessed in the future for trad-\ning decision-making. For example, a straightforward trad-\ning decision can be buy/sell/hold stocks when the predicted\nclass is up/down/ﬂat, respectively. Given that CTTS’s pre-\ndicted probablitlies are reliable as discussed earlier, the\namount of stock shares to buy/sell/hold can depend on the\n\nand 4

In [15]:
query = "summarize the paper, please."
qa_chain_with_memory({"query": query})

Token indices sequence length is longer than the specified maximum sequence length for this model (4873 > 4096). Running this sequence through the model will result in indexing errors
/root/miniconda3/envs/llm-tutorial/lib/python3.9/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (4096) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Input length of input_ids is 4873, but `max_length` is set to 4096. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant for question-answering tasks. Use the following pieces of retrieved context and answer the question at the end.
If you don't know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.
Context= work, autoregression and recurrent neural networks (Hastie,
Tibshirani, and Friedman 2001). More recent works have ap-
plied Deep learning (DL) methods on numeric time series
data (Bao, Yue, and Rao 2017; Gensler et al. 2016; Romeu
et al. 2015; Sagheer and Kotb 2019; Sutskever, Vinyals, and
Le 2014). DL automates the process of feature extraction
and eliminates the need for domain expertise.
Since the introduction of transformers (Vaswani et al.
2017), they have become the state of the art model to im-arXiv:2304.04912v1  [cs.LG]  11 Apr 2023

and long-term dependencies within

{'query': 'summarize the paper, please.',
 'result': 'You are an assistant for question-answering tasks. Use the following pieces of retrieved context and answer the question at the end.\nIf you don\'t know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.\nContext= work, autoregression and recurrent neural networks (Hastie,\nTibshirani, and Friedman 2001). More recent works have ap-\nplied Deep learning (DL) methods on numeric time series\ndata (Bao, Yue, and Rao 2017; Gensler et al. 2016; Romeu\net al. 2015; Sagheer and Kotb 2019; Sutskever, Vinyals, and\nLe 2014). DL automates the process of feature extraction\nand eliminates the need for domain expertise.\nSince the introduction of transformers (Vaswani et al.\n2017), they have become the state of the art model to im-arXiv:2304.04912v1  [cs.LG]  11 Apr 2023\n\nand long-term dependencies within a time series, and forecast\nif the pr

In [7]:
# query = "这篇文章的主题是什么\n"
# docs = docsearch.get_relevant_documents(query)
# print("-"*20+"number of relevant documents"+"-"*20)
# print(len(docs))

--------------------number of relevant documents--------------------
4
